Those as follow are good references for learning database/python interfaces. <br />
https://www.datacamp.com/community/tutorials/mysql-python <br />
https://www.techonthenet.com/mariadb/

In [2]:
import pandas
import mariadb
import sys

In [3]:
# connect to mariadb server, it is optional to set up db. One can 
# create new one or modify existing one. 
# using the command of "mysql -u root -p -h localhost" to log in mariadb in your terminal

try:
    conn = mariadb.connect(
        host="localhost",
        user="root", 
        passwd="56691003")
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)
    
# create interface between python and mariadb server using cursor

cur = conn.cursor()


# after the connection, we can use python code to constrct or modify maria databases
# by usnig .execute()
# One can refer to https://www.techonthenet.com/mariadb/index.php for sql commands.
# Using the command of "show databases; " to check the existence of databases. 
# Please do not forget the ';'.

cur.execute("create database if not exists test_db \
             character set = 'utf8mb4' \
             collate = 'utf8mb4_general_ci'")

#import csv files using pandas

df = pandas.read_csv("aftergoogle.csv", sep=',', index_col = 0) 


# create table in your db, for example test_db in this case
# using the commands of "use database_name; (you need to specifiy the name)" to log in the database.
# then, using the command of "describe table_name" to see tables in the related database


table_name = 'test_db.test_table'
cur.execute("create or replace table test_db.test_table \
            (empty_var text not null);") 

# create column in table, namely test_table here
for column in df:
    cur.execute(f"alter table test_db.test_table \
                add {column} text not null;")


    
cur.execute(f'alter table {table_name} drop column empty_var')
# add data into corresponding columns in the created table

row_num = df.shape[0]
table_name = 'test_db.test_table'

for n in range(row_num):
    lst = list(df.iloc[1,:].astype('str'))
    lst = ['\"' + n + '\"' for n in lst]
    
    cur.execute("insert into {table_name} ({columns})values ({values})".format(
                table_name = table_name, 
                columns = ', '.join(list(df.columns)),  
                values = ', '.join(lst)))

# finally using the commend of select * from table_name to see your input data, 
# after successfully executing conn.commit()
conn.commit()


cur.close()